In [3]:
import sympy as sym
import numpy as np
import pandas as pd

# Alle Variablen:

In [4]:
T_G_ein, T_G_aus = sym.symbols('T_G_ein T_G_aus')

p_U, phi_U, T_U, T_FF_aus, p_aus, phi_aus = sym.symbols('p_U phi_U T_U T_FF_aus p_aus phi_aus')

T_KW_ein, T_KW_aus = sym.symbols('T_KW_ein T_KW_aus')

M_KW, d_i, d_a, L = sym.symbols('M_KW d_i d_a L')

lambda_Wand, pi = sym.symbols('lambda_Wand pi')

M_G_aus, M_Verdampfer, M_K_aus = sym.symbols('M_G_aus M_Verdampfer M_K_aus')

# Funktionsdefinition

## Berechnungen und Stoffwerte

In [24]:
h_L_ein = 1013.3*T_G_ein-641.82

h_L_aus = 1006.1*T_G_aus-29.407
    
h_D_ein = 1899.48*T_G_ein+2498539.75
    
h_D_aus = 1873.72*T_G_aus+2500723
    
h_K_aus = 4179.5*(T_KW_ein+5)+361.44
    
eta_KW = 2.34*10**(-7)*T_KW_ein**2-3.08*10**(-5)*T_KW_ein+1.51*10**(-3)
    
nü_KW = 2.33*10**(-10)*T_KW_ein**2-3.06*10**(-8)*T_KW_ein+1.51*10**(-6)
    
lambda_KW = 0.0015*T_KW_ein+0.5703
    
rho_KW = -0.3429*T_KW_ein+1006
    
cp_KW = 0.0123*T_KW_ein**2-1.0074*T_KW_ein+4199.1

A_i = d_i * pi * L

A_a = d_a * pi * L

X_ein = 0.622*(0.00611657*sym.exp(17.2799-4102.99/(237.431+T_U)))/(p_U/phi_U-0.00611657*sym.exp(17.2799-4102.99/(237.431+T_U)))

X_aus =  0.622*(0.00611657*sym.exp(17.2799-4102.99/(237.431+T_FF_aus)))/(p_aus/phi_aus-0.00611657*sym.exp(17.2799-4102.99/(237.431+T_FF_aus)))

DeltaT_m = ((T_G_ein-T_KW_aus)-(T_G_aus - T_KW_ein))/(sym.log((T_G_ein-T_KW_aus)/((T_G_aus - T_KW_ein))))

Q = M_G_aus/(1+X_aus)*(h_L_ein-h_L_aus)+(M_G_aus/(1+X_aus)*X_ein+M_Verdampfer)*h_D_ein-M_G_aus/(1/X_aus+1)*h_D_aus-M_K_aus*h_K_aus

A_Q = d_i**2 / 4 * pi

w = M_KW/rho_KW/A_Q

Re = w*d_a/nü_KW

Pr = eta_KW *cp_KW /lambda_KW 

alpha_KW = 1/(0.79*sym.log(Re )-1.64)**2/8*Re *Pr /(1+12.7*(1/(0.79*sym.log(Re )-1.64)**2/8)**0.5*(Pr **(2/3)-1))*(1+(d_a/L)**(2/3))*lambda_KW/d_a

alpha_i = 1/(A_i*(DeltaT_m/Q -sym.log(d_a/d_i)/(lambda_Wand*2*pi*L)-1/(alpha_KW*A_a)))


## Ableitungen

In [6]:
dp_U = sym.diff(alpha_i, p_U)

dT_U = sym.diff(alpha_i, T_U)

dp_aus = sym.diff(alpha_i,p_aus)

dT_G_ein = sym.diff(alpha_i, T_G_ein)

dT_G_aus = sym.diff(alpha_i, T_G_aus)

dphi_U = sym.diff(alpha_i, phi_U)

dphi_aus = sym.diff(alpha_i, phi_aus)

dT_FF_aus = sym.diff(alpha_i, T_FF_aus)

dT_KW_ein = sym.diff(alpha_i, T_KW_ein)

dT_KW_aus = sym.diff(alpha_i, T_KW_aus)

dM_G_aus = sym.diff(alpha_i, M_G_aus)

dM_Verdampfer = sym.diff(alpha_i, M_Verdampfer)

dM_K_aus = sym.diff(alpha_i, M_K_aus)

dM_KW = sym.diff(alpha_i, M_KW)


## Automatisierung

Absolute Fehler berechnen

In [7]:
def Abs_Fehler(Daten):
    Names = ['p_U','phi_U','T_U','T_G_ein','p_aus','phi_aus','T_FF_aus','T_G_aus','T_KW_ein','T_KW_aus','M_Verdampfer','M_KW','M_K_aus','M_G_aus']
    date = np.array([[Daten.loc[0].p_U*3/100,Daten.loc[0].phi_U*1.3/100,0.2,0.3,Daten.loc[0].p_aus*0.15/100*2,Daten.loc[0].phi_aus*1.3/100,0.2,0.3,0.3,0.3,Daten.loc[0].M_Verdampfer*1.3/100,Daten.loc[0].M_KW*4/100,Daten.loc[0].M_K_aus*8/100,Daten.loc[0].M_G_aus*0.05/100]])

    Fehler = pd.DataFrame(date, columns = Names)
    return Fehler

In [16]:
def Berechnung_Fehler(Daten):
        
    # Berechnung der absolutne Fehler der Messwerte    
    Fehler = Abs_Fehler(Daten)

    #Erstellen des Frames fuer die Speicherung der Berechneten Fehler
    NamesF = ['p_U','phi_U','T_U','T_G_ein','p_aus','phi_aus','T_FF_aus','T_G_aus','T_KW_ein','T_KW_aus','M_Verdampfer','M_KW','M_K_aus','M_G_aus']

    dFehler = pd.DataFrame(columns = NamesF)
    dFehler.loc[0] = 0
    
    # Berechnung der Fehler
    
    dFehler.loc[0].p_U = Fehler.loc[0].p_U * dp_U.subs(M_G_aus, Daten.M_G_aus).subs(p_U, Daten.p_U).subs(p_aus,Daten.p_aus).subs(phi_aus,Daten.phi_aus).subs(T_G_aus,Daten.T_G_aus).subs(T_FF_aus,Daten.T_FF_aus).subs(phi_U,Daten.phi_U).subs(T_G_ein,Daten.T_G_ein).subs(M_Verdampfer,Daten.M_Verdampfer).subs(M_K_aus,Daten.M_K_aus).subs(T_U, Daten.T_U).subs(T_KW_ein,Daten.T_KW_ein).subs(T_KW_ein, Daten.T_KW_ein).subs(M_KW, Daten.M_KW).subs(d_a, Daten.d_a).subs(d_i,Daten.d_i).subs(L,Daten.L).subs(pi, Daten.pi).subs(lambda_Wand, Daten.lambda_Wand).subs(T_KW_aus,Daten.T_KW_aus)
    
    dFehler.loc[0].T_U = Fehler.loc[0].T_U* dT_U.subs(M_G_aus, Daten.M_G_aus).subs(p_U, Daten.p_U).subs(p_aus,Daten.p_aus).subs(phi_aus,Daten.phi_aus).subs(T_G_aus,Daten.T_G_aus).subs(T_FF_aus,Daten.T_FF_aus).subs(phi_U,Daten.phi_U).subs(T_G_ein,Daten.T_G_ein).subs(M_Verdampfer,Daten.M_Verdampfer).subs(M_K_aus,Daten.M_K_aus).subs(T_U, Daten.T_U).subs(T_KW_ein,Daten.T_KW_ein).subs(T_KW_ein, Daten.T_KW_ein).subs(M_KW, Daten.M_KW).subs(d_a, Daten.d_a).subs(d_i,Daten.d_i).subs(L,Daten.L).subs(pi, Daten.pi).subs(lambda_Wand, Daten.lambda_Wand).subs(T_KW_aus,Daten.T_KW_aus)
    
    dFehler.loc[0].p_aus = Fehler.loc[0].p_aus* dp_aus.subs(M_G_aus, Daten.M_G_aus).subs(p_U, Daten.p_U).subs(p_aus,Daten.p_aus).subs(phi_aus,Daten.phi_aus).subs(T_G_aus,Daten.T_G_aus).subs(T_FF_aus,Daten.T_FF_aus).subs(phi_U,Daten.phi_U).subs(T_G_ein,Daten.T_G_ein).subs(M_Verdampfer,Daten.M_Verdampfer).subs(M_K_aus,Daten.M_K_aus).subs(T_U, Daten.T_U).subs(T_KW_ein,Daten.T_KW_ein).subs(T_KW_ein, Daten.T_KW_ein).subs(M_KW, Daten.M_KW).subs(d_a, Daten.d_a).subs(d_i,Daten.d_i).subs(L,Daten.L).subs(pi, Daten.pi).subs(lambda_Wand, Daten.lambda_Wand).subs(T_KW_aus,Daten.T_KW_aus)
    
    dFehler.loc[0].T_G_ein = Fehler.loc[0].T_G_ein* dT_G_ein.subs(M_G_aus, Daten.M_G_aus).subs(p_U, Daten.p_U).subs(p_aus,Daten.p_aus).subs(phi_aus,Daten.phi_aus).subs(T_G_aus,Daten.T_G_aus).subs(T_FF_aus,Daten.T_FF_aus).subs(phi_U,Daten.phi_U).subs(T_G_ein,Daten.T_G_ein).subs(M_Verdampfer,Daten.M_Verdampfer).subs(M_K_aus,Daten.M_K_aus).subs(T_U, Daten.T_U).subs(T_KW_ein,Daten.T_KW_ein).subs(T_KW_ein, Daten.T_KW_ein).subs(M_KW, Daten.M_KW).subs(d_a, Daten.d_a).subs(d_i,Daten.d_i).subs(L,Daten.L).subs(pi, Daten.pi).subs(lambda_Wand, Daten.lambda_Wand).subs(T_KW_aus,Daten.T_KW_aus)
    
    dFehler.loc[0].T_G_aus = Fehler.loc[0].T_G_aus* dT_G_aus.subs(M_G_aus, Daten.M_G_aus).subs(p_U, Daten.p_U).subs(p_aus,Daten.p_aus).subs(phi_aus,Daten.phi_aus).subs(T_G_aus,Daten.T_G_aus).subs(T_FF_aus,Daten.T_FF_aus).subs(phi_U,Daten.phi_U).subs(T_G_ein,Daten.T_G_ein).subs(M_Verdampfer,Daten.M_Verdampfer).subs(M_K_aus,Daten.M_K_aus).subs(T_U, Daten.T_U).subs(T_KW_ein,Daten.T_KW_ein).subs(T_KW_ein, Daten.T_KW_ein).subs(M_KW, Daten.M_KW).subs(d_a, Daten.d_a).subs(d_i,Daten.d_i).subs(L,Daten.L).subs(pi, Daten.pi).subs(lambda_Wand, Daten.lambda_Wand).subs(T_KW_aus,Daten.T_KW_aus)
    
    dFehler.loc[0].phi_U = Fehler.loc[0].phi_U* dphi_U.subs(M_G_aus, Daten.M_G_aus).subs(p_U, Daten.p_U).subs(p_aus,Daten.p_aus).subs(phi_aus,Daten.phi_aus).subs(T_G_aus,Daten.T_G_aus).subs(T_FF_aus,Daten.T_FF_aus).subs(phi_U,Daten.phi_U).subs(T_G_ein,Daten.T_G_ein).subs(M_Verdampfer,Daten.M_Verdampfer).subs(M_K_aus,Daten.M_K_aus).subs(T_U, Daten.T_U).subs(T_KW_ein,Daten.T_KW_ein).subs(T_KW_ein, Daten.T_KW_ein).subs(M_KW, Daten.M_KW).subs(d_a, Daten.d_a).subs(d_i,Daten.d_i).subs(L,Daten.L).subs(pi, Daten.pi).subs(lambda_Wand, Daten.lambda_Wand).subs(T_KW_aus,Daten.T_KW_aus)
    
    dFehler.loc[0].phi_aus = Fehler.loc[0].phi_aus* dphi_aus.subs(M_G_aus, Daten.M_G_aus).subs(p_U, Daten.p_U).subs(p_aus,Daten.p_aus).subs(phi_aus,Daten.phi_aus).subs(T_G_aus,Daten.T_G_aus).subs(T_FF_aus,Daten.T_FF_aus).subs(phi_U,Daten.phi_U).subs(T_G_ein,Daten.T_G_ein).subs(M_Verdampfer,Daten.M_Verdampfer).subs(M_K_aus,Daten.M_K_aus).subs(T_U, Daten.T_U).subs(T_KW_ein,Daten.T_KW_ein).subs(T_KW_ein, Daten.T_KW_ein).subs(M_KW, Daten.M_KW).subs(d_a, Daten.d_a).subs(d_i,Daten.d_i).subs(L,Daten.L).subs(pi, Daten.pi).subs(lambda_Wand, Daten.lambda_Wand).subs(T_KW_aus,Daten.T_KW_aus)
    
    dFehler.loc[0].T_FF_aus = Fehler.loc[0].T_FF_aus* dT_FF_aus.subs(M_G_aus, Daten.M_G_aus).subs(p_U, Daten.p_U).subs(p_aus,Daten.p_aus).subs(phi_aus,Daten.phi_aus).subs(T_G_aus,Daten.T_G_aus).subs(T_FF_aus,Daten.T_FF_aus).subs(phi_U,Daten.phi_U).subs(T_G_ein,Daten.T_G_ein).subs(M_Verdampfer,Daten.M_Verdampfer).subs(M_K_aus,Daten.M_K_aus).subs(T_U, Daten.T_U).subs(T_KW_ein,Daten.T_KW_ein).subs(T_KW_ein, Daten.T_KW_ein).subs(M_KW, Daten.M_KW).subs(d_a, Daten.d_a).subs(d_i,Daten.d_i).subs(L,Daten.L).subs(pi, Daten.pi).subs(lambda_Wand, Daten.lambda_Wand).subs(T_KW_aus,Daten.T_KW_aus)
    
    dFehler.loc[0].T_KW_ein = Fehler.loc[0].T_KW_ein* dT_KW_ein.subs(M_G_aus, Daten.M_G_aus).subs(p_U, Daten.p_U).subs(p_aus,Daten.p_aus).subs(phi_aus,Daten.phi_aus).subs(T_G_aus,Daten.T_G_aus).subs(T_FF_aus,Daten.T_FF_aus).subs(phi_U,Daten.phi_U).subs(T_G_ein,Daten.T_G_ein).subs(M_Verdampfer,Daten.M_Verdampfer).subs(M_K_aus,Daten.M_K_aus).subs(T_U, Daten.T_U).subs(T_KW_ein,Daten.T_KW_ein).subs(T_KW_ein, Daten.T_KW_ein).subs(M_KW, Daten.M_KW).subs(d_a, Daten.d_a).subs(d_i,Daten.d_i).subs(L,Daten.L).subs(pi, Daten.pi).subs(lambda_Wand, Daten.lambda_Wand).subs(T_KW_aus,Daten.T_KW_aus)
    
    dFehler.loc[0].T_KW_aus = Fehler.loc[0].T_KW_aus* dT_KW_aus.subs(M_G_aus, Daten.M_G_aus).subs(p_U, Daten.p_U).subs(p_aus,Daten.p_aus).subs(phi_aus,Daten.phi_aus).subs(T_G_aus,Daten.T_G_aus).subs(T_FF_aus,Daten.T_FF_aus).subs(phi_U,Daten.phi_U).subs(T_G_ein,Daten.T_G_ein).subs(M_Verdampfer,Daten.M_Verdampfer).subs(M_K_aus,Daten.M_K_aus).subs(T_U, Daten.T_U).subs(T_KW_ein,Daten.T_KW_ein).subs(T_KW_ein, Daten.T_KW_ein).subs(M_KW, Daten.M_KW).subs(d_a, Daten.d_a).subs(d_i,Daten.d_i).subs(L,Daten.L).subs(pi, Daten.pi).subs(lambda_Wand, Daten.lambda_Wand).subs(T_KW_aus,Daten.T_KW_aus)
    
    dFehler.loc[0].M_G_aus = Fehler.loc[0].M_G_aus* dM_G_aus.subs(M_G_aus, Daten.M_G_aus).subs(p_U, Daten.p_U).subs(p_aus,Daten.p_aus).subs(phi_aus,Daten.phi_aus).subs(T_G_aus,Daten.T_G_aus).subs(T_FF_aus,Daten.T_FF_aus).subs(phi_U,Daten.phi_U).subs(T_G_ein,Daten.T_G_ein).subs(M_Verdampfer,Daten.M_Verdampfer).subs(M_K_aus,Daten.M_K_aus).subs(T_U, Daten.T_U).subs(T_KW_ein,Daten.T_KW_ein).subs(T_KW_ein, Daten.T_KW_ein).subs(M_KW, Daten.M_KW).subs(d_a, Daten.d_a).subs(d_i,Daten.d_i).subs(L,Daten.L).subs(pi, Daten.pi).subs(lambda_Wand, Daten.lambda_Wand).subs(T_KW_aus,Daten.T_KW_aus)
    
    dFehler.loc[0].M_Verdampfer = Fehler.loc[0].M_Verdampfer* dM_Verdampfer.subs(M_G_aus, Daten.M_G_aus).subs(p_U, Daten.p_U).subs(p_aus,Daten.p_aus).subs(phi_aus,Daten.phi_aus).subs(T_G_aus,Daten.T_G_aus).subs(T_FF_aus,Daten.T_FF_aus).subs(phi_U,Daten.phi_U).subs(T_G_ein,Daten.T_G_ein).subs(M_Verdampfer,Daten.M_Verdampfer).subs(M_K_aus,Daten.M_K_aus).subs(T_U, Daten.T_U).subs(T_KW_ein,Daten.T_KW_ein).subs(T_KW_ein, Daten.T_KW_ein).subs(M_KW, Daten.M_KW).subs(d_a, Daten.d_a).subs(d_i,Daten.d_i).subs(L,Daten.L).subs(pi, Daten.pi).subs(lambda_Wand, Daten.lambda_Wand).subs(T_KW_aus,Daten.T_KW_aus)
    
    dFehler.loc[0].M_K_aus = Fehler.loc[0].M_K_aus* dM_K_aus.subs(M_G_aus, Daten.M_G_aus).subs(p_U, Daten.p_U).subs(p_aus,Daten.p_aus).subs(phi_aus,Daten.phi_aus).subs(T_G_aus,Daten.T_G_aus).subs(T_FF_aus,Daten.T_FF_aus).subs(phi_U,Daten.phi_U).subs(T_G_ein,Daten.T_G_ein).subs(M_Verdampfer,Daten.M_Verdampfer).subs(M_K_aus,Daten.M_K_aus).subs(T_U, Daten.T_U).subs(T_KW_ein,Daten.T_KW_ein).subs(T_KW_ein, Daten.T_KW_ein).subs(M_KW, Daten.M_KW).subs(d_a, Daten.d_a).subs(d_i,Daten.d_i).subs(L,Daten.L).subs(pi, Daten.pi).subs(lambda_Wand, Daten.lambda_Wand).subs(T_KW_aus,Daten.T_KW_aus)
    
    dFehler.loc[0].M_KW = Fehler.loc[0].M_KW* dM_KW.subs(M_G_aus, Daten.M_G_aus).subs(p_U, Daten.p_U).subs(p_aus,Daten.p_aus).subs(phi_aus,Daten.phi_aus).subs(T_G_aus,Daten.T_G_aus).subs(T_FF_aus,Daten.T_FF_aus).subs(phi_U,Daten.phi_U).subs(T_G_ein,Daten.T_G_ein).subs(M_Verdampfer,Daten.M_Verdampfer).subs(M_K_aus,Daten.M_K_aus).subs(T_U, Daten.T_U).subs(T_KW_ein,Daten.T_KW_ein).subs(T_KW_ein, Daten.T_KW_ein).subs(M_KW, Daten.M_KW).subs(d_a, Daten.d_a).subs(d_i,Daten.d_i).subs(L,Daten.L).subs(pi, Daten.pi).subs(lambda_Wand, Daten.lambda_Wand).subs(T_KW_aus,Daten.T_KW_aus)
    
    dFehler.loc[0] = abs(dFehler.loc[0])
    
    Alpha_innen = alpha_i.subs(M_G_aus, Daten.M_G_aus).subs(p_U, Daten.p_U).subs(p_aus,Daten.p_aus).subs(phi_aus,Daten.phi_aus).subs(T_G_aus,Daten.T_G_aus).subs(T_FF_aus,Daten.T_FF_aus).subs(phi_U,Daten.phi_U).subs(T_G_ein,Daten.T_G_ein).subs(M_Verdampfer,Daten.M_Verdampfer).subs(M_K_aus,Daten.M_K_aus).subs(T_U, Daten.T_U).subs(T_KW_ein,Daten.T_KW_ein).subs(T_KW_ein, Daten.T_KW_ein).subs(M_KW, Daten.M_KW).subs(d_a, Daten.d_a).subs(d_i,Daten.d_i).subs(L,Daten.L).subs(pi, Daten.pi).subs(lambda_Wand, Daten.lambda_Wand).subs(T_KW_aus,Daten.T_KW_aus)
    
    ErgebnisDaten = np.array([[Alpha_innen, sum(dFehler.loc[0]), sum(dFehler.loc[0])/Alpha_innen * 100]] )    
    
    Werte = ['Alpha_gemessen','Alpha_Fehler_Absolut','Alpha_Fehler_Relativ']
    
    Ergebnis = pd.DataFrame(ErgebnisDaten, columns = Werte)
    
    return Ergebnis

Messdaten noch manuel

In [21]:
Namen = ['T_G_ein','T_G_aus','p_U','phi_U','T_U','T_FF_aus','p_aus','phi_aus','T_KW_ein','T_KW_aus','M_KW','d_i','d_a','L','lambda_Wand','pi','M_G_aus','M_Verdampfer','M_K_aus']
data = np.array([[120.1083118,46.17755222,0.987,25.87330322/100,22.7035297080504,39.85311404,1.08526318275,98.77265298/100,35.13196451,35.50382756,0.8295,0.02,0.022,3.040,20,np.pi,0.009701786091,8.318*10**(-4),0.000461583]])
data.T

Daten = pd.DataFrame(data,columns = Namen)

In [28]:
Berechnung_Fehler(Daten)


,Alpha_gemessen,Alpha_Fehler_Absolut,Alpha_Fehler_Relativ
0,289.513996651150,18.2345937075823,6.29834616581737
